# Gathering candlestick data
For fetching historical candlestick data using the bitfinex api. The main function used is fetch_hist which can be found in [bitfinex_api.py](https://github.com/darren-the/Algorithmic-trader/blob/master/bitfinex_api.py)

In [5]:
from bitfinex_api import fetch_hist
from utils import tLength
import copy
import csv

## Setting parameters
The parameters to pass into fetch_hist for fetching historical candlestick data
* path: the path to save the data to (str)
* symbol: the symbol we want information about (str). E.g. 'BTCUSD', 'ETHUSD', 'USD', 'BTC'
* timeframe: (str) '1m', '5m', '15m', '30m', '1h', '3h', '6h', '12h', '1D', '7D', '14D'
* start: the starting timestamp in ms (float)
* end: the ending timestamp in ms (float)

In [2]:
path = 'test_ochl.csv'
symbol = 'BTCUSD'
timeframe = '15m'
start = 1483228800000
end = 1598918400000
ochl_data_raw = fetch_hist(symbol, timeframe, start, end)

100%|██████████| 100/100 [00:22&lt;00:00,  4.43it/s]


## Dealing with missing values
There will most likely be some missing candlesticks. Here we will iterate through the data, keeping track of the correct timestamp. If a candlestick is found to be missing, it will be filled in with a duplicate of the next available candlestick.

In [6]:
real_t = start
i = 0
ochl_data = []
while i < len(ochl_data_raw) or real_t < end:
    while real_t < ochl_data_raw[i][0]:
        if i == 0:
            new_ochl = copy.copy(ochl_data_raw[i+1]) 
        else:
            new_ochl = copy.copy(ochl_data_raw[i-1])
        new_ochl[0] = real_t
        ochl_data.append(new_ochl)
        real_t += tLength[timeframe]

    ochl_data.append(ochl_data_raw[i])
    if i < len(ochl_data_raw):
        i += 1
    real_t += tLength[timeframe]

## Saving data

In [15]:
with open(path, 'w') as f:
    writer = csv.writer(f)
    writer.writerows(ochl_data)